# 看完后可以学到什么

- 学会继承collections模块中Iterable和Iterator类来自定义自己的Iterable和Iterator
- 学会使用yield关键字来实现生成器
- 学会实现`__reversed__`来提供反向迭代的接口
- 原来python中文本文件是可迭代对象
- 学会使用itertools中的islice方法对迭代器切片（以往我们都是对列表进行切片的）
- 学会用zip实现在一个for语句中并行迭代多个可迭代对象
- 学会用itertools中的chain函数实现在一个for语句中串行迭代多个可迭代对象

# 如何实现可迭代对象和迭代器对象

In [9]:
from collections import Iterable, Iterator

Iterable.__abstractmethods__, Iterator.__abstractmethods__

(frozenset({'__iter__'}), frozenset({'next'}))

Iterable的抽象方法是`__iter__`，Iterator的抽象方法是`next`

In [7]:
class WeatherIterator(Iterator):
    
    def __init__(self, cities):
        self.cities = cities
        self.index = 0
        
    def getWeather(self, city):
        return city + "....."
        
    def next(self):
        if self.index == len(self.cities):
            raise StopIteration
        city = self.cities[self.index]
        self.index += 1
        return self.getWeather(city)

class WeatherIterable(Iterable):
    
    def __init__(self, cities):
        self.cities = cities
        
    def __iter__(self):
        return WeatherIterator(self.cities)

for x in WeatherIterable(['北京', '武汉']):
    print(x)

北京.....
武汉.....


# 如何使用生成器函数实现可迭代对象
- 实现一个可迭代对象的类, 它能迭代出给定范围内所有素数

In [14]:
from math import sqrt

class PrimeNumber:
    
    def __init__(self, start, end):
        self.start = start
        self.end = end
        
    def isPrime(self, k):
        for i in range(2, int(sqrt(k) + 1)): # math.sqrt
            if k % i == 0:
                return False
            return True
        
    def __iter__(self):
        for k in range(self.start, self.end+1):
            if self.isPrime(k):
                yield k  #关键点

for x in PrimeNumber(1, 15):
    print(x)

5
7
9
11
13
15


# 如何进行反向迭代以及如何实现反向迭代
- 实现一个连续的浮点数发生器, 根据给定的范围和步长产生一系列的浮点数(反向)

In [21]:
class FloatRange:
    
    def __init__(self, start, end, step=0.1):
        self.start = start
        self.end = end
        self.step = step
        
    def __iter__(self):
        t = self.start
        while t <= self.end:
            yield t
            t += self.step
            
    def __reversed__(self):
        """reversed函数以该类为参数输入时会调用该函数"""
        t = self.end
        while t >= self.start:
            yield t
            t -= self.step
      
    
print list(FloatRange(1, 5, 0.5))
print list(reversed(FloatRange(1, 5, 0.5)))

[1, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]
[5, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0]


# 如何对迭代器做切片操作
- 有某个文本文件, 我们想读取其中某范围的内容, 如100~300行之间的内容( python中文本文件是可迭代对象)

In [40]:
f = open("data/words-en.txt")
f.readlines()

['\n',
 'Python\n',
 'Python logo and wordmark.svg\n',
 'Paradigm\tMulti-paradigm: functional, imperative, object-oriented, reflective\n',
 'Designed by\tGuido van Rossum\n',
 'Developer\tPython Software Foundation\n',
 'First appeared\t1990; 29 years ago[1]\n',
 'Stable release\t\n',
 '3.7.2 / 24 December 2018; 23 days ago[2]\n',
 '2.7.15 / 1 May 2018; 8 months ago[3]\n',
 'Typing discipline\tDuck, dynamic, gradual (since 3.5),[4] strong\n',
 'License\tPython Software Foundation License\n',
 'Filename extensions\t.py, .pyc, .pyd, .pyo (prior to 3.5),[5] .pyw, .pyz (since 3.5)[6]\n',
 'Website\twww.python.org\n',
 'Major implementations\n',
 'CPython, IronPython, Jython, MicroPython, Numba, PyPy, Stackless Python, CircuitPython\n',
 'Dialects\n',
 'Cython, RPython\n',
 'Influenced by\n',
 'ABC,[7] ALGOL 68,[8] APL[9] C,[10] C++,[11] CLU,[12] Dylan,[13] Haskell,[14] Icon,[15] Java,[16] Lisp,[17] Modula-3,[11] Perl, Standard ML[9]\n',
 'Influenced\n',
 'Boo, Cobra, CoffeeScript,[18] D, F

In [41]:
f.seek(0)

In [42]:
from itertools import islice
a = islice(f, 5, 15) # 5-51行，若要读到末尾，则15可写成None
list(a)

['Developer\tPython Software Foundation\n',
 'First appeared\t1990; 29 years ago[1]\n',
 'Stable release\t\n',
 '3.7.2 / 24 December 2018; 23 days ago[2]\n',
 '2.7.15 / 1 May 2018; 8 months ago[3]\n',
 'Typing discipline\tDuck, dynamic, gradual (since 3.5),[4] strong\n',
 'License\tPython Software Foundation License\n',
 'Filename extensions\t.py, .pyc, .pyd, .pyo (prior to 3.5),[5] .pyw, .pyz (since 3.5)[6]\n',
 'Website\twww.python.org\n',
 'Major implementations\n']

**注意islice会消耗原来的迭代对象，比如上面调用了islice(f, 5, 15)，那么在再次迭代f的后就是从15行之后开始的，在使用迭代器的时候需要注意这一点。**

In [43]:
[i for i in f]

['CPython, IronPython, Jython, MicroPython, Numba, PyPy, Stackless Python, CircuitPython\n',
 'Dialects\n',
 'Cython, RPython\n',
 'Influenced by\n',
 'ABC,[7] ALGOL 68,[8] APL[9] C,[10] C++,[11] CLU,[12] Dylan,[13] Haskell,[14] Icon,[15] Java,[16] Lisp,[17] Modula-3,[11] Perl, Standard ML[9]\n',
 'Influenced\n',
 'Boo, Cobra, CoffeeScript,[18] D, F#, Genie,[19] Go, Apache Groovy, JavaScript,[20][21] Julia,[22] Nim, Ring,[23] Ruby,[24] Swift[25]\n',
 ' Python Programming at Wikibooks\n',
 'Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python has a design philosophy that emphasizes code readability, notably using significant whitespace. It provides constructs that enable clear programming on both small and large scales.[26] Van Rossum led the language community until stepping down as leader in July 2018.[27][28]\n',
 '\n',
 'Python features a dynamic type system and automatic memory management. It supp

# 如何在一个 for 语句中迭代多个可迭代对象
- 并行迭代：学生各科成绩分别存储在各个对应列表中, 同时迭代这些列表, 计算每个学生的总成绩

In [47]:
chinese = [80,70,60,90]
math = [65,78,86,96]
english= [75,85,63,97]
for a, b,c in zip(chinese, math, english):
    print a+b+c

152
153
231
249


- 串行迭代：某年级4个班, 某次英语考试成绩存在4个列表中, 统计成绩高于90的人数

In [51]:
from itertools import chain

english1 = [75, 85, 63, 97]
english2 = [75, 85, 63, 97]
english3 = [75, 85, 63, 97]
english4 = [75, 85, 63, 97]
cnt = 0
for i in chain(english1, english2, english3, english4):
    if i > 90:
        cnt +=1
cnt

4